
# CNN em Física: Classificação de Fases no Modelo de Ising 2D (Keras)

## Alexandre Suaide

Este notebook usa uma **Rede Neural Convolucional (CNN)** para distinguir **duas fases** do modelo de **Ising 2D** a partir de imagens de spins (matrizes 16×16 com valores ±1).
- **Fase ferromagnética (baixa T)** vs **fase paramagnética (alta T)**.
- Usamos o dataset **Ising Light** do **MDS-2** (`mdsdata`). Se não estiver disponível, o notebook **gera dados sintéticos** semelhantes para fins didáticos.


In [ ]:

# Se estiver no Google Colab, descomente a linha abaixo para instalar dependências:
# !pip install mdsdata tensorflow scikit-learn seaborn joblib --quiet


In [ ]:
# importa os módulos necessários
# dependendo da versão do TensoFlow, aparece um monte de warnings

import os, sys, math, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import joblib

# Reprodutibilidade do gerador de números aleatórios

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


In [ ]:
# Carregamento do dataset Ising Light (MDS-2) 

def load_ising_mds2_or_synthetic(n_samples_synth=5000, L=16):
    from mdsdata import load_Ising_light
    X, y, temp = load_Ising_light()  # X: (N,16,16) com spins ±1 ; y: 0 (ferro), 1 (para)
    print("Dataset MDS-2 Ising Light carregado:", X.shape, y.shape)
    return X.astype(np.float32), y.astype(np.int64)
    

X, y = load_ising_mds2_or_synthetic()
labels_map = {0: "Ferromagnético (baixa T)", 1: "Paramagnético (alta T)"}

In [ ]:
# Faz algumas figuras aleatórias para visualização

fig, axes = plt.subplots(4, 5, figsize=(12, 10))
for i, ax in enumerate(axes.flat):
    idx = np.random.randint(0, len(X))
    ax.imshow(X[idx], cmap="coolwarm", interpolation="nearest")
    ax.set_title(labels_map[int(y[idx])], fontsize=10)
    ax.axis("off")
plt.suptitle("Configurações de spins do Ising 2D", fontsize=14)
plt.show()


In [ ]:
# Preparação dos dados para CNN

# Adicionar canal (necessário para Conv2D) -> (N, 16, 16, 1)
X = X[..., None].astype(np.float32)

# Normalizar: spins (-1,+1) -> (0,1) para estabilidade numérica
X = (X + 1.0) / 2.0

# divisão do dataset em treino, teste e validação. 
# as variáveis abaixo determinam as frações

train_ratio = 0.50
test_ratio = 0.25
validation_ratio = 0.25

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, stratify = y, random_state = RANDOM_SEED)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify = y_test, random_state = RANDOM_SEED) 

print(f"train = {len(X_train)} \ntest = {len(X_test)} \nval = {len(X_val)}")


In [ ]:
# Construção do modelo no Keras

model = keras.Sequential([
    layers.Input((16,16,1)),
    layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(2, activation="softmax")
])

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


In [ ]:
# treinamento da rede
# verbose = 0, 1 ou 2 indica a quantidade de print
# aqui são usadas duas estratégias de saída

early_stop = keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor="val_loss")
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=4, verbose=1, monitor="val_loss")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=128,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


In [ ]:
# faz figura da perda e da acurácia para cada época

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('Perda')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title('Acurácia')
plt.legend()
plt.show()


In [ ]:
# Avaliar o dataset de teste
# obtém a matriz de confusão

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Acurácia no teste: {test_acc:.4f}")

y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred, target_names=[labels_map[0], labels_map[1]]))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=[labels_map[0], labels_map[1]], yticklabels=[labels_map[0], labels_map[1]], cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()


In [ ]:
# Fazer predições e printar o resultado

n_show = 8
idxs = np.random.choice(len(X_test), size=n_show, replace=False)
fig, axes = plt.subplots(2, n_show//2, figsize=(12, 5))
for k, ax in enumerate(axes.flat):
    i = idxs[k]
    img = X_test[i,...,0] * 2.0 - 1.0  # voltar para escala (-1,1) só para visualização
    pred = y_pred[i]
    true = y_test[i]
    ax.imshow(img, cmap="coolwarm", interpolation="nearest")
    ax.set_title(f"P:{labels_map[int(pred)].split()[0]} / T:{labels_map[int(true)].split()[0]}", fontsize = 8)
    ax.axis("off")
plt.suptitle("Amostras de teste com Predição (P) e Verdadeiro (T)")
plt.show()


In [ ]:
# salva o modelo para uso futuro em novos dados. Dai não precisa treinar a rede novamente

model.save("ising_cnn_keras.keras")
print("Modelo salvo como ising_cnn_keras.keras")
